In [ ]:
import torch
import pickle
import numpy as np
from argparse import Namespace
from torch.utils.data import DataLoader
import torch.nn.functional as F
from heterogt.utils.tokenizer import EHRTokenizer
from heterogt.utils.dataset import FineTuneEHRDataset, batcher, expand_level3
from heterogt.utils.train import train_with_early_stopping
from heterogt.utils.seed import set_random_seed
from heterogt.model.model import HeteroGTFineTune

In [ ]:
set_random_seed(123)

[INFO] Random seed set to 123


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
config = Namespace(
    dataset = "MIMIC-IV",
    tasks = ["death", "readmission", "stay", "next_diag_6m", "next_diag_12m"], 
    task_index = 0,  # index of the task to train
    token_type = ["diag", "med", "lab", "pro"],
    special_tokens = ["[PAD]", "[CLS]"],
    attn_mask_dicts = [{1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 6:[2,3,4,5,6,7], 7:[2,3,4,5,6,7]}, 
                       {1:[6,7], 2:[6,7], 3:[6,7], 4:[6,7], 6:[2,3,4,5,6,7], 7:[2,3,4,5,6,7]}],
    d_model = 64,
    num_heads = 4,
    batch_size = 32,
    lr = 1e-3,
    epochs = 500,
    early_stop_patience = 5,
    group_code_thre = 5,  # if there are group_code_thre diag codes belongs to the same group ICD code, then the group code is generated
    use_pretrained_model = True,
    pretrain_mask_rate = 0.7,
    pretrain_cls_ontology_weight = 5e-2,
    pretrain_visit_ontology_weight = 5e-2,
    pretrain_adm_type_weight = 5e-2,
)

In [ ]:
full_data_path = f"./data_process/{config.dataset}-processed/mimic.pkl"  # for tokenizer
curr_task = config.tasks[config.task_index]
print("Current task:", curr_task)
if curr_task == "next_diag_6m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_6m.pkl"
elif curr_task == "next_diag_12m":
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_nextdiag_12m.pkl"
else:
    finetune_data_path = f"./data_process/{config.dataset}-processed/mimic_downstream.pkl"

Current task: stay


In [ ]:
ehr_full_data = pickle.load(open(full_data_path, 'rb'))
group_code_sentences = [expand_level3()[1]]
diag_sentences = ehr_full_data["ICD9_CODE"].values.tolist()
med_sentences = ehr_full_data["NDC"].values.tolist()
lab_sentences = ehr_full_data["LAB_TEST"].values.tolist()
pro_sentences = ehr_full_data["PRO_CODE"].values.tolist()
age_sentences = [[str(c)] for c in set(ehr_full_data["AGE"].values.tolist())] # important of [[]]
adm_type_sentences = ehr_full_data["ADMISSION_TYPE"].values.tolist()
max_admissions = ehr_full_data.groupby("SUBJECT_ID")["HADM_ID"].nunique().max()
config.max_num_adms = max_admissions
print(f"Max admissions per patient: {config.max_num_adms}")

Max admissions per patient: 8


In [ ]:
tokenizer = EHRTokenizer(age_sentences, group_code_sentences, diag_sentences, med_sentences, lab_sentences, 
                         pro_sentences, special_tokens=config.special_tokens, adm_types_sentences=adm_type_sentences)
config.label_vocab_size = len(tokenizer.diag_voc.id2word)  # only for diagnosis
config.global_vocab_size = len(tokenizer.vocab.id2word)
config.age_vocab_size = tokenizer.token_number("age")
config.group_code_vocab_size = tokenizer.token_number("group")
print(f"Age vocabulary size: {config.age_vocab_size}")
print(f"Group code vocabulary size: {config.group_code_vocab_size}")

Age vocabulary size: 20
Group code vocabulary size: 19


In [ ]:
train_data, val_data, test_data = pickle.load(open(finetune_data_path, 'rb'))
# example label percentage
print("Percentage of DEATH in test dataset:",
      (test_data["DEATH"] == True).mean() * 100, "%")

print("Percentage of READMISSION in test dataset:",
      (test_data["READMISSION"] == 1).mean() * 100, "%")

print("Percentage of STAY>7 days in test dataset:",
      (test_data["STAY_DAYS"] > 7).mean() * 100, "%")

Percentage of DEATH in test dataset: 9.077523846104603 %
Percentage of READMISSION in test dataset: 70.78932206644902 %
Percentage of STAY>7 days in test dataset: 33.36534152743103 %


In [ ]:
train_dataset = FineTuneEHRDataset(train_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
val_dataset = FineTuneEHRDataset(val_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                 max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)
test_dataset = FineTuneEHRDataset(test_data, tokenizer, token_type=config.token_type, task=curr_task, 
                                   max_num_adms=config.max_num_adms, group_code_thre=config.group_code_thre)

In [ ]:
num_group_code = []
for i in range(len(train_dataset)):
    input_ids, token_types, adm_index, age_ids, diag_group_codes, labels = train_dataset[i]
    count = (token_types[0] == 6).sum().item()
    num_group_code.append(count)
print("Mean group token numer per patient", np.mean(num_group_code))

Mean group token numer per patient 1.0251935441543105


In [ ]:
train_dataloader = DataLoader(
    train_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=True,
    batch_size=config.batch_size,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=batcher(tokenizer, n_token_type=len(config.token_type), is_pretrain = False),
    shuffle=False,
    batch_size=config.batch_size,
)

In [ ]:
for batch in train_dataloader:
    pass  # just to check if the dataloader works
for batch in val_dataloader:
    pass  # just to check if the dataloader works
for batch in test_dataloader:
    pass  # just to check if the dataloader works
print("All pass!")

All pass!


In [ ]:
if curr_task in ["death", "stay", "readmission"]:
    eval_metric = "f1"
    task_type = "binary"
    loss_fn = F.binary_cross_entropy_with_logits
else:
    eval_metric = "f1"
    task_type = "l2r"
    loss_fn = lambda x, y: F.binary_cross_entropy_with_logits(x, y)

In [ ]:
input_ids, token_types, adm_index, age_ids, diag_code_group_dicts, labels = next(iter(train_dataloader))
print("Input IDs shape:", input_ids.shape)
print("Token Types shape:", token_types.shape)
print("Admission Index shape:", adm_index.shape)
print("Age IDs shape:", age_ids.shape)
print("Diag Code Group Dict number:", len(diag_code_group_dicts))
print("Labels shape:", labels.shape)

Input IDs shape: torch.Size([32, 242])
Token Types shape: torch.Size([32, 242])
Admission Index shape: torch.Size([32, 242])
Age IDs shape: torch.Size([32, 6])
Diag Code Group Dict number: 32
Labels shape: torch.Size([32, 1])


# Model Walkthrough

In [ ]:
# load pretrained model
if config.use_pretrained_model:
    pretrain_exp_name = (
    f"{config.dataset}-{config.pretrain_mask_rate}-{config.d_model}-{config.pretrain_cls_ontology_weight}-{config.pretrain_visit_ontology_weight}-{config.pretrain_adm_type_weight}"
)
    print(pretrain_exp_name)
    save_path = "./pretrained_models/" + pretrain_exp_name
    state_dict = torch.load(f"{save_path}/pretrained_model.pt", map_location="cpu")

MIMIC-IV-0.7-64-0.05-0.05-0.05


In [ ]:
final_metrics = []
for i in range(15):
    model = HeteroGTFineTune(tokenizer=tokenizer, token_types=config.token_type, d_model=config.d_model, num_heads=config.num_heads, layer_types=['gnn', 'tf', 'gnn', 'tf'], max_num_adms=config.max_num_adms, 
                     device=device, task=curr_task, label_vocab_size=config.label_vocab_size, attn_mask_dicts=config.attn_mask_dicts,
                     use_cls_cat=True).to(device)
    if config.use_pretrained_model:
        model.load_weight(state_dict)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config.lr)
    best_test_metric = train_with_early_stopping(model, train_dataloader, val_dataloader, test_dataloader,
                                             optimizer, loss_fn, device, config.early_stop_patience, task_type, config.epochs, 
                                             val_long_seq_idx=None, test_long_seq_idx=None, eval_metric=eval_metric, return_model=False)
    final_metrics.append(best_test_metric)

[Warning] Missing keys: ['cls_head.cls.0.weight', 'cls_head.cls.0.bias', 'cls_head.cls.2.weight', 'cls_head.cls.2.bias']
[Warning] Unexpected keys: ['diag_cls.cls.0.weight', 'diag_cls.cls.0.bias', 'diag_cls.cls.2.weight', 'diag_cls.cls.2.bias', 'med_cls.cls.0.weight', 'med_cls.cls.0.bias', 'med_cls.cls.2.weight', 'med_cls.cls.2.bias', 'lab_cls.cls.0.weight', 'lab_cls.cls.0.bias', 'lab_cls.cls.2.weight', 'lab_cls.cls.2.bias', 'pro_cls.cls.0.weight', 'pro_cls.cls.0.bias', 'pro_cls.cls.2.weight', 'pro_cls.cls.2.bias', 'cls_ontology.cls.0.weight', 'cls_ontology.cls.0.bias', 'cls_ontology.cls.2.weight', 'cls_ontology.cls.2.bias', 'visit_ontology.cls.0.weight', 'visit_ontology.cls.0.bias', 'visit_ontology.cls.2.weight', 'visit_ontology.cls.2.bias', 'adm_type.cls.0.weight', 'adm_type.cls.0.bias', 'adm_type.cls.2.weight', 'adm_type.cls.2.bias']


Running inference: 100%|██████████| 489/489 [00:22<00:00, 21.29it/s]


Validation: {'precision': 0.7303276393146728, 'recall': 0.5009604302717999, 'f1': 0.5942805010397599, 'auc': 0.8365172735198022, 'prauc': 0.7291904376143467}
Test:      {'precision': 0.7239227340245946, 'recall': 0.4673829623935776, 'f1': 0.5680307752235438, 'auc': 0.8277744914501124, 'prauc': 0.7081596535546029}


Running inference: 100%|██████████| 489/489 [00:21<00:00, 22.40it/s]


Validation: {'precision': 0.6334769427148021, 'recall': 0.76258163657172, 'f1': 0.6920596132756693, 'auc': 0.8572341360376237, 'prauc': 0.7623224498750384}
Test:      {'precision': 0.6247419406056459, 'recall': 0.7547966231758351, 'f1': 0.683638886345709, 'auc': 0.8518142046329675, 'prauc': 0.7444143822729752}


...

Validation: {'precision': 0.7080016757421282, 'recall': 0.6492508643859983, 'f1': 0.677354704426849, 'auc': 0.8561333199686408, 'prauc': 0.766362003428819}
Test:      {'precision': 0.6898484533927967, 'recall': 0.6375671527232587, 'f1': 0.6626782281303752, 'auc': 0.849992271211074, 'prauc': 0.7520950378208544}


Running inference: 100%|██████████| 489/489 [00:21<00:00, 22.81it/s]


Validation: {'precision': 0.6571678321666833, 'recall': 0.7220514790612331, 'f1': 0.6880834656303444, 'auc': 0.852804823799905, 'prauc': 0.7603954614697175}
Test:      {'precision': 0.6437239944510338, 'recall': 0.7123944742887329, 'f1': 0.6763205778895489, 'auc': 0.8466539892163395, 'prauc': 0.7450051018503541}


Running inference: 100%|██████████| 489/489 [00:21<00:00, 22.70it/s]

Validation: {'precision': 0.6852760736182305, 'recall': 0.6436803688039884, 'f1': 0.6638272533237102, 'auc': 0.8472639654286552, 'prauc': 0.7523229957091657}
Test:      {'precision': 0.6801470588221402, 'recall': 0.6389102072128954, 'f1': 0.6588840472394357, 'auc': 0.8429415254539083, 'prauc': 0.7410949790529747}

Early stopping triggered (no improvement for 5 epochs).

Best validation performance:
{'precision': 0.6346307864798421, 'recall': 0.7610449481353034, 'f1': 0.6921128433235036, 'auc': 0.8596651974925374, 'prauc': 0.7731665611430583}
Corresponding test performance:
{'precision': 0.6231123827680447, 'recall': 0.7521105141965616, 'f1': 0.6815613268692888, 'auc': 0.8527011997484024, 'prauc': 0.754721086173909}


In [ ]:
def topk_avg_performance_formatted(performances, k=5):
    metrics = ["f1", "auc", "prauc"]
    scores = {m: np.array([p[m] for p in performances]) for m in metrics}

    # 计算排名（值越大排名越靠前）
    ranks = {m: (-scores[m]).argsort().argsort() + 1 for m in metrics}
    avg_ranks = np.mean(np.stack([ranks[m] for m in metrics], axis=1), axis=1)

    # 选 top-k
    topk_idx = np.argsort(avg_ranks)[:k]
    final_avg = {m: np.mean([performances[i][m] for i in topk_idx]) for m in performances[0].keys()}
    final_std = {m: np.std([performances[i][m] for i in topk_idx], ddof=0) for m in performances[0].keys()}

    # 打印结果（转百分比，均保留两位小数）
    print("Final Metrics:")
    for m in performances[0].keys():
        mean_val = final_avg[m] * 100
        std_val = final_std[m] * 100
        print(f"{m}: {mean_val:.2f} ± {std_val:.2f}")

In [ ]:
topk_avg_performance_formatted(final_metrics, 5)

Final Metrics:
precision: 62.22 ± 1.07
recall: 76.43 ± 2.00
f1: 68.57 ± 0.37
auc: 85.29 ± 0.16
prauc: 75.12 ± 0.43
